In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.naive_bayes import BernoulliNB
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('weatherAUS.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['MinTemp'] = df['MinTemp'].fillna(df['MinTemp'].mean())
df['MaxTemp'] = df['MaxTemp'].fillna(df['MaxTemp'].mean())
df['Rainfall'] = df['Rainfall'].fillna(df['Rainfall'].mean())
df['Evaporation'] = df['Evaporation'].fillna(df['Evaporation'].mean())
df['Sunshine'] = df['Sunshine'].fillna(df['Sunshine'].mean())
df['WindGustSpeed'] = df['WindGustSpeed'].fillna(df['WindGustSpeed'].mean())
df['WindSpeed9am'] = df['WindSpeed9am'].fillna(df['WindSpeed9am'].mean())
df['WindSpeed3pm'] = df['WindSpeed3pm'].fillna(df['WindSpeed3pm'].mean())
df['Humidity9am'] = df['Humidity9am'].fillna(df['Humidity9am'].mean())
df['Humidity3pm'] = df['Humidity3pm'].fillna(df['Humidity3pm'].mean())
df['Pressure9am'] = df['Pressure9am'].fillna(df['Pressure9am'].mean())
df['Pressure3pm'] = df['Pressure3pm'].fillna(df['Pressure3pm'].mean())
df['Cloud9am'] = df['Cloud9am'].fillna(df['Cloud9am'].mean())
df['Cloud3pm'] = df['Cloud3pm'].fillna(df['Cloud3pm'].mean())


In [ ]:
df['WindGustDir'] = df['WindGustDir'].fillna(df['WindGustDir'].mode()[0])
df['WindDir9am'] = df['WindDir9am'].fillna(df['WindDir9am'].mode()[0])
df['WindDir3pm'] = df['WindDir3pm'].fillna(df['WindDir3pm'].mode()[0])
df['Temp9am'] = df['Temp9am'].fillna(df['Temp9am'].mode()[0])
df['Temp3pm'] = df['Temp3pm'].fillna(df['Temp3pm'].mode()[0])
df['RainToday'] = df['RainToday'].fillna(df['RainToday'].mode()[0])
df['RainTomorrow'] = df['RainTomorrow'].fillna(df['RainTomorrow'].mode()[0])

In [ ]:
df.drop_duplicates()

In [ ]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df[['year','month','day']].head()


In [ ]:
df = df.drop(['Date','year'], axis=1)


In [ ]:
df1 = df.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm','WindGustDir','WindGustSpeed','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Temp9am','Temp3pm'], axis = 1)
df1.head()


In [ ]:
encoder = LabelEncoder()
df1['Location'] = encoder.fit_transform(df1['Location'])
df1['RainToday'] = encoder.fit_transform(df1['RainToday'])
df1['RainTomorrow'] = encoder.fit_transform(df1['RainTomorrow'])

In [ ]:
q1, q3 = np.percentile(df1['Rainfall'], (25, 75))
iqr = q3 - q1
min = q1 - 1.5*iqr
max = q3 + 1.5*iqr
outliers_indecies = df1[df1['Rainfall'] > max].index.tolist()
df1 = df1.drop(outliers_indecies, axis=0)
df1.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
df2 = df1.copy()

In [ ]:

column = 'MaxTemp'
df2[column] = MinMaxScaler().fit_transform(np.array(df2[column]).reshape(-1,1))

In [ ]:
display(df2)

In [ ]:
x = df2.drop(['RainTomorrow'], axis=1)
y = df2['RainTomorrow']


In [ ]:
x_trn,x_tst,y_trn,y_tst = train_test_split(x,y,test_size = 0.2,shuffle = True, random_state = 55)
print(x_trn.shape)
print(x_tst.shape)
print(y_trn.shape)
print(y_tst.shape)


In [ ]:
clf = GradientBoostingClassifier(n_estimators = 50,learning_rate = 0.5)
clf.fit(x_trn,y_trn)

In [ ]:
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))

In [ ]:
#pip install virtualenv


In [ ]:
pip install -U flask-cors

In [ ]:
from flask import Flask,render_template,url_for,request,jsonify
from flask_cors import cross_origin
import pandas as pd
import numpy as np
import datetime
import pickle



#app = Flask(_name_)
app=Flask(__name__,template_folder='templates')
model = pickle.load(open("model.pkl", "rb"))
print("Model Loaded")

@app.route("/",methods=['GET'])
@cross_origin()
def home():
    return render_template("predictor1.html")
    
@app.route("/predict",methods=['GET', 'POST'])
@cross_origin()
def predict():
    if request.method == "POST":
            date = request.form['date']
            day = float(pd.to_datetime(date, format="%Y-%m-%dT").day)
            month = float(pd.to_datetime(date, format="%Y-%m-%dT").month)
            minTemp = float(request.form['mintemp'])
            maxTemp = float(request.form['maxtemp'])
            rainfall = float(request.form['rainfall'])
            location = float(request.form['location'])
            rainToday = float(request.form['raintoday'])
            input_lst = [location , minTemp , maxTemp , rainfall,
                     rainToday , month , day]
            pred = model.predict(input_lst)
            output = pred
            if output == 0:
                    return render_template("after_sunny1.html")
            else:
                    return render_template("after_rainy1.html")

if __name__=='__main__':
   # app.run(debug=True, port=8051)
    app.run(debug=True, use_reloader=False)